In [ ]:
# Imports
#%matplotlib widget
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib
import h5py
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
from diffpy.pdfgetx.pdfconfig import PDFConfig
from diffpy.pdfgetx.pdfgetter import PDFGetter
from diffpy.morph.morphpy import morph_arrays
from diffpy.utils.parsers import load_data
from ufpdf_xfel_scripts.lcls.paths import synchrotron_data_dir, experiment_data_dir, global_output_dir
from ufpdf_xfel_scripts.lcls.run import Run
from ufpdf_xfel_scripts.lcls.plots import (plot_delay_scans,
                                           plot_static_scans,
                                           plot_reference_comparison,
                                           plot_gr_function)

import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger("diffpy.pdfgetx").setLevel(logging.ERROR)
logging.getLogger("diffpy.pdfgetx.user").setLevel(logging.ERROR)
logging.getLogger().setLevel(logging.ERROR)

In [ ]:
# Experiment: LCLS 202601
instrument = 'mfx'
experiment_number = 'l1044925'

In [ ]:
# User: Luis
username = 'lkitsu'

In [ ]:
# Setup:

# setup for roi
q_min =2.5 # I(q) figure of merit (fom)
q_max = 3 # I(q) fom
r_min_fom = 2 # G(r) fom 
r_max_fom = 5 # G(r) fom

# global setup for morphs
target_id = 0 #index of the delay_time Iq_off to use as target for morphing as well as calculate adhoc corrections for g(r)

# setup for morph I(q) delay
q_min_morph = 0 #qmin for normalization. None uses the smaller value. Not working for new diffpy.morph! Also is called xmin now.
q_max_morph = 12 #qmax for normalization. None uses the largest value. Not working for new diffpy.morph! Also is called xmax now.
scale = 1.01 
stretch = None
smear = None
baselineslope = None
qdamp = None
params = {'xmin': q_min_morph, 'xmax': q_max_morph, 'scale': scale,'stretch': stretch, 'smear':smear}
t0 = 0
points_away_t0_plot_on_off = 0

# setup for morph F(q)
fit_qmin = 0.1
fit_qmax = 10
pdf_rmin = 0
pdf_rmax = 60
initial_guesses = {'rpoly': 1, 'qmin': 0.1, 'bgscale': 1}
squeeze = [0,0,0,0,0]

# setup for pdfgetx3
getter_config = {
    "rpoly": 1.,
    "qmin": 0.5,
    "qmax": 10.,
    "qmaxinst": 12.,
    "bgscale": None,     # set to none to suppress background correction
}

In [ ]:
# Run:
run_number = 58
background_number = 1
sample_name = 'NSPSe'
sample_composition = 'Na11SnPSe12'
number_of_static_samples = 11   # only needed for static runs, will be ignored otherwise
verbose = False
plot_monitor_normalizations = False
# delay motor name used for this run
delay_motor = "mfx_lxt_fast2"
# delay_motor = "lxt_ttx"

In [ ]:
# Instantiate run object
run_object = Run(run_number,
                 background_number,
                 sample_name,
                 sample_composition,
                 instrument,
                 experiment_number,
                 number_of_static_samples=number_of_static_samples,
                 delay_motor=delay_motor,
                 getter_config=getter_config,
                 squeeze=squeeze,
                 )
morph_parameters = run_object.morph_parameters # Dictionary with fitted morph parameters. The key is the delay.

In [ ]:
# Plot Iq
if run_object.delay_scan:
    plot_delay_scans(run_object.morphed_delay_scans, run_object)
    if plot_monitor_normalizations:
        plot_delay_scans(run_object.mon1_normalized_delay_scans, run_object)
        plot_delay_scans(run_object.mon2_normalized_delay_scans, run_object)
else:
    plot_static_scans(run_object.morphed_static_scans, run_object)


In [ ]:
# Plot f(Q)
if run_object.delay_scan:
    plot_delay_scans(run_object.fq_delay_scans, run_object)

In [ ]:
# Plot F(r)
if run_object.delay_scan:
    plot_delay_scans(run_object.gr_delay_scans, run_object)

In [ ]:
plot_reference_comparison(
    q_target=run_object.q_synchrotron,
    fq_target=run_object.fq_synchrotron,
    q_morph=run_object.fq_delay_scans[:, 0],
    fq_morph=run_object.fq_delay_scans[:, 1],
    r_min=0,
    r_max=30,
)

In [ ]:
plot_gr_function(
    run_object.gr_delay_scans,
    sample_name=sample_name,
    run_number=run_number,
    r_min_fom=r_min_fom,
    r_max_fom=r_max_fom
)